In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import joblib

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/food_inference_dataset_100000.csv")
df.head(10)

,name,allergies,diet,conditions,ingredients,Calories,Total Fat,Saturated Fat,Sodium,Total Carbohydrate,Sugar,Protein,should_consume
0,User4761,gluten,keto,NaN,"Coconut,Shrimp,Peanuts,Soy Lecithin,Beef,Almonds",232,10.0,1.5,676,20.7,18.1,2.5,Yes
1,User8703,shellfish,diabetic,lactose intolerance,"Sesame,Egg,Chicken,Palm Oil,Cheese,Almonds,Shr...",276,19.0,4.7,775,17.6,3.4,13.3,Yes
2,User3403,NaN,vegetarian,hypertension,"Soy Lecithin,Onion,Tomato,Honey,Almonds,Chili,...",405,2.7,9.0,523,59.7,4.2,11.8,No
3,User369,"dairy,soy",diabetic,"obesity,lactose intolerance","Corn,Wheat,Onion,Milk,Oats,Sesame,Tomato,Butte...",225,16.1,5.5,746,66.0,8.4,9.8,Yes
4,User2832,shellfish,paleo,high cholesterol,"Coconut,Egg,Palm Oil,Milk,Fish,Butter,Sugar,Sa...",395,13.8,5.4,1179,18.4,19.8,18.9,Yes
5,User9430,NaN,keto,lactose intolerance,"Fish,Chicken,Beef,Sugar,Sesame,Wheat,Mustard,E...",403,22.4,4.2,256,23.3,10.5,20.9,Yes
6,User9183,peanuts,keto,heart disease,"Sugar,Tomato,Wheat,Honey,Garlic",347,19.3,0.2,1078,20.3,15.5,19.0,Yes
7,User1323,"peanuts,soy",vegan,hypertension,"Sesame,Tomato,Yogurt,Beef,Chili,Honey,Fish,Shr...",198,4.8,8.2,178,32.5,20.8,13.3,No
8,User6363,"eggs,peanuts",paleo,"heart disease,obesity","Celery,Chicken,Fish,Shrimp,Milk,Wheat,Peanuts",399,1.9,4.1,127,60.6,16.4,21.8,No
9,User6435,NaN,diabetic,"lactose intolerance,high cholesterol","Shrimp,Sesame,Milk,Chicken,Almonds,Fish,Garlic...",129,11.4,9.4,1121,69.6,12.5,20.4,Yes


In [ ]:
print("\nData info:")
print(df.info())


Data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   name                100000 non-null  object 
 1   allergies           66736 non-null   object 
 2   diet                100000 non-null  object 
 3   conditions          66850 non-null   object 
 4   ingredients         100000 non-null  object 
 5   Calories            100000 non-null  int64  
 6   Total Fat           100000 non-null  float64
 7   Saturated Fat       100000 non-null  float64
 8   Sodium              100000 non-null  int64  
 9   Total Carbohydrate  100000 non-null  float64
 10  Sugar               100000 non-null  float64
 11  Protein             100000 non-null  float64
 12  should_consume      100000 non-null  object 
dtypes: float64(5), int64(2), object(6)
memory usage: 9.9+ MB
None


In [ ]:
print("\nSummary statistics:")
print(df.describe(include='all'))


Summary statistics:
            name  allergies        diet conditions  \
count     100000      66736      100000      66850   
unique     10000         49           5         25   
top     User3088  shellfish  vegetarian    obesity   
freq          24       4854       20259       6728   
mean         NaN        NaN         NaN        NaN   
std          NaN        NaN         NaN        NaN   
min          NaN        NaN         NaN        NaN   
25%          NaN        NaN         NaN        NaN   
50%          NaN        NaN         NaN        NaN   
75%          NaN        NaN         NaN        NaN   
max          NaN        NaN         NaN        NaN   

                                ingredients       Calories      Total Fat  \
count                                100000  100000.000000  100000.000000   
unique                                99991            NaN            NaN   
top     Garlic,Palm Oil,Wheat,Butter,Barley            NaN            NaN   
freq                  

In [ ]:
df['allergies'] = df['allergies'].fillna('none')
df['conditions'] = df['conditions'].fillna('none')

In [ ]:
X = df.drop(columns=['name', 'should_consume'])
y = df['should_consume'].map({'Yes': 1, 'No': 0})

In [ ]:
X

,allergies,diet,conditions,ingredients,Calories,Total Fat,Saturated Fat,Sodium,Total Carbohydrate,Sugar,Protein
0,gluten,keto,none,"Coconut,Shrimp,Peanuts,Soy Lecithin,Beef,Almonds",232,10.0,1.5,676,20.7,18.1,2.5
1,shellfish,diabetic,lactose intolerance,"Sesame,Egg,Chicken,Palm Oil,Cheese,Almonds,Shr...",276,19.0,4.7,775,17.6,3.4,13.3
2,none,vegetarian,hypertension,"Soy Lecithin,Onion,Tomato,Honey,Almonds,Chili,...",405,2.7,9.0,523,59.7,4.2,11.8
3,"dairy,soy",diabetic,"obesity,lactose intolerance","Corn,Wheat,Onion,Milk,Oats,Sesame,Tomato,Butte...",225,16.1,5.5,746,66.0,8.4,9.8
4,shellfish,paleo,high cholesterol,"Coconut,Egg,Palm Oil,Milk,Fish,Butter,Sugar,Sa...",395,13.8,5.4,1179,18.4,19.8,18.9
...,...,...,...,...,...,...,...,...,...,...,...
99995,"shellfish,tree nuts",paleo,obesity,"Oats,Honey,Mustard,Garlic,Barley,Celery,Shrimp...",441,10.2,4.1,874,31.4,9.3,17.9
99996,none,diabetic,obesity,"Fish,Olive Oil,Peanuts,Tomato,Yogurt,Barley,Co...",205,3.0,8.4,864,52.7,0.8,2.9
99997,dairy,vegetarian,"obesity,heart disease","Sugar,Chicken,Chili,Cheese,Fish,Beef,Onion",309,24.4,4.4,537,47.5,18.2,15.9
99998,none,keto,hypertension,"Barley,Onion,Palm Oil,Tomato,Olive Oil,Spinach",409,6.7,4.5,1039,18.1,20.1,3.9


In [ ]:
y

,should_consume
0,1
1,1
2,0
3,1
4,1
...,...
99995,1
99996,1
99997,0
99998,0


In [ ]:
categorical_cols = ['allergies', 'diet', 'conditions', 'ingredients']
numerical_cols = ['Calories', 'Total Fat', 'Saturated Fat', 'Sodium',
                  'Total Carbohydrate', 'Sugar', 'Protein']

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='none')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ('cat', categorical_transformer, categorical_cols),
    ('num', numerical_transformer, numerical_cols)
])

In [ ]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:54:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='none',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['allergies', 'diet',
                                                   'conditions',
                                                   'ingredients']),
                                                 ('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Calories', 'Total Fat',
                                                   'Saturated Fat', 'Sodiu...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [ ]:
y_pred = model.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.95      0.91      8555
           1       0.96      0.89      0.92     11445

    accuracy                           0.92     20000
   macro avg       0.91      0.92      0.91     20000
weighted avg       0.92      0.92      0.92     20000



In [ ]:
joblib.dump(model, "food_consumption_model_xgb.pkl")
print("\nModel saved to food_consumption_model_xgb.pkl")


Model saved to food_consumption_model_xgb.pkl
